<a href="https://colab.research.google.com/github/Madhan30d/511323106030-cracking-the-market-code/blob/main/511323106030_cracking_the_market_coe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Step 1: Load Stock Data
def load_data(stock_symbol='AAPL', start='2018-01-01', end='2024-12-31'):
    df = yf.download(stock_symbol, start=start, end=end)
    return df['Close'].values.reshape(-1, 1), df

# Step 2: Preprocess Data
def preprocess_data(data, sequence_length=60):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []
    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i])
        y.append(scaled_data[i])

    return np.array(X), np.array(y), scaler

# Step 3: Build LSTM Model
def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Step 4: Train and Predict
def train_and_predict(stock_symbol='AAPL'):
    raw_data, full_df = load_data(stock_symbol)
    X, y, scaler = preprocess_data(raw_data)

    split = int(len(X) * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    model = build_model((X_train.shape[1], 1))
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Plot Results
    plt.figure(figsize=(14, 6))
    plt.plot(y_test_actual, color='blue', label='Actual Stock Price')
    plt.plot(predictions, color='red', label='Predicted Stock Price')
    plt.title(f'{stock_symbol} Stock Price Prediction with LSTM')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.show()

# Run it
train_and_predict('AAPL')  # Change to any stock symbol like 'GOOGL', 'MSFT', etc.
